# Create a CAIC weather data map 

## 1. Install requirements 

In [1]:
%pip install folium datetime


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## 2. Import requirements

In [2]:
import folium
import datetime
from folium.plugins import GroupedLayerControl
from folium.plugins import MeasureControl
from folium.plugins import MousePosition

## 3. Generate Functions
- The first function collections the current date and time for use later on url creation. 
- The second function generates urls that are the base url for different CAIC products.

In [3]:
def get_current_datetime():
    now = datetime.datetime.now()
    return now.strftime("%Y-%m-%d+%H:%M")

In [4]:
def generate_station_url(station_code, station_title, elevation, url_type):
    current_datetime = get_current_datetime()
    title_encoded = station_title.replace(" ", "+")
    
    if url_type == 'weather':
        return f"https://classic.avalanche.state.co.us/caic/obs_stns/station.php?plot=hourly&st={station_code}&date={current_datetime}&unit=e&area=caic&title={title_encoded}"
    elif url_type == 'windrose':
        return f"https://classic.avalanche.state.co.us/caic/obs_stns/windrose.php?st={station_code}&date={current_datetime}&elev={elevation}&unit=e&area=caic&title={title_encoded}"
    elif url_type == 'plot':
        return f"https://classic.avalanche.state.co.us/caic/obs_stns/hplot.php?title={title_encoded}&st={station_code}&date={current_datetime}&unit=e&area=caic&range=48"
    else:
        raise ValueError(f"Invalid URL type: {url_type}")

## 4. Define Folium function to generate the map

In [5]:
def weather_map(stations, forecast_locations, webcam_locations, center_lat=39.5501, center_lon=-106.0667, zoom_start=9):

    m = folium.Map(location=[center_lat, center_lon], zoom_start=zoom_start, control_scale=True)

    # Define a list of tile layers with their configurations
    tile_layers = [
        {
            'tiles': 'openstreetmap',
            'name': 'OpenStreetMap',
            'attr': None  # No additional attribution needed for OpenStreetMap
        },
        {
            'tiles': 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            'name': 'Esri World Imagery',
            'attr': 'Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community'
        },
        {
            'tiles': 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/{z}/{y}/{x}',
            'name': 'Esri World Topo Map',
            'attr': 'Tiles &copy; Esri &mdash; Esri, DeLorme, NAVTEQ, TomTom, Intermap, iPC, USGS, FAO, NPS, NRCAN, GeoBase, Kadaster NL, Ordnance Survey, Esri Japan, METI, Esri China (Hong Kong), and the GIS User Community'
        }
    ]

    # Loop through the tile list and add each tile layer to the map
    for layer in tile_layers:
        if layer['tiles'] == 'openstreetmap':
            folium.TileLayer(layer['tiles'], name=layer['name']).add_to(m)
        else:
            folium.TileLayer(
                tiles=layer['tiles'],
                name=layer['name'],
                attr=layer['attr']
            ).add_to(m)
            
    # Set Group Layers - This breaks the map objects into groups which can be selected for visibilty later with group layer control
    stations_group = folium.FeatureGroup(name='Weather Stations')
    forecast_group = folium.FeatureGroup(name='Forecast Locations')
    webcam_group = folium.FeatureGroup(name='Webcams')

    # Add weather stations - Pulls from stations list to build map objects 
    # Stations will display links to current weather, current wind rose (link will display even if n/a), a current graph plot, and link to graph plot
    for station in stations:
        if station['latitude'] and station['longitude']:
            weather_url = generate_station_url(station['code'], station['title'], station['elevation'], 'weather')
            windrose_url = generate_station_url(station['code'], station['title'], station['elevation'], 'windrose')
            plot_url = generate_station_url(station['code'], station['title'], station['elevation'], 'plot')
            
            popup_content = f"""
            <div style="width:300px;">
                <b>{station['name']}</b><br>
                Provider: {station['provider']}<br>
                Lat: {station['latitude']}, Lon: {station['longitude']}<br>
                Elevation: {station['elevation']} ft<br>
                <a href="{weather_url}" target="_blank">Current Weather</a><br>
                <a href="{windrose_url}" target="_blank">Current Windrose</a><br>
                <img src="{plot_url}" alt="Weather Plot" style="width:100%; max-width:100%;"><br>
                <a href="{plot_url}" target="_blank">Open Full Weather Plot</a>
            </div>
            """
            
            folium.Marker(
                location=[station['latitude'], station['longitude']],
                popup=folium.Popup(popup_content, max_width=300),
                tooltip=station['name'],
                icon=folium.Icon(color='blue', icon='info-sign')
            ).add_to(stations_group)

    # Add forecast locations - Pulls from forecast dictionary 
    # Forecast graphs have a current static link that is linked directly for three models 
    for location in forecast_locations:
        popup_content = f"""
        <div>
            <b>{location['name']}</b><br>
            Lat: {location['latitude']}, Lon: {location['longitude']}<br>
            Model Elevation: {location['model_elevation']} ft<br>
            Forecast Images:<br>
            <a href="{location['forecast_image_urls']['WRF']}" target="_blank">WRF</a> |
            <a href="{location['forecast_image_urls']['WRFHR']}" target="_blank">WRFHR</a> |
            <a href="{location['forecast_image_urls']['NAM']}" target="_blank">NAM</a>
        </div>
        """

        folium.Marker(
            location=[location['latitude'], location['longitude']],
            popup=folium.Popup(popup_content, max_width=300),
            tooltip=location['name'],
            icon=folium.Icon(color='red', icon='cloud')
        ).add_to(forecast_group)

    # Add webcam locations - Pulls from webcam dictionary 
    # Displays image and link to static webcams 
    for name, webcam in webcam_locations.items():
        popup_content = f"""
        <div style="width:400px; height:400px;">
            <b>{name}</b><br>
            Lat: {webcam['latitude']}, Lon: {webcam['longitude']}<br>
            <a href="{webcam['image_url']}" target="_blank">
                <img src="{webcam['image_url']}" alt="{name} Webcam"
                     style="width:100%; height:300px; object-fit:cover; cursor:pointer;"
                     title="Click to open full-size image">
            </a>
            <p><small>Click the image to open in full size</small></p>
        </div>
        """

        folium.Marker(
            location=[webcam['latitude'], webcam['longitude']],
            popup=folium.Popup(popup_content, max_width=400),
            tooltip=name,
            icon=folium.Icon(color='green', icon='camera')
        ).add_to(webcam_group)

    # Add group layers to the map
    stations_group.add_to(m)
    forecast_group.add_to(m)
    webcam_group.add_to(m)

    # Add layer controls
    folium.LayerControl(collapsed=False).add_to(m)

    # Add grouped layer control
    GroupedLayerControl(
        groups={
            'Map Layers': [stations_group, forecast_group, webcam_group]
        },
        exclusive_groups=False,
        collapsed=False
    ).add_to(m)

    title_html = '''
             <h3 align="center" style="font-size:16px"><b>CAIC Weather Stations, Forecast Locations and Webcams</b></h3>
             '''
    m.get_root().html.add_child(folium.Element(title_html))

    # Add measure tool
    m.add_child(MeasureControl())

    # Add mouse position
    MousePosition().add_to(m)

    return m

## 5. Object Data 
- Weather station data and url codes
- Forecast locations and direct urls
- Webcam locations and direct urls

In [6]:
# List of stations 
stations = [
    {"name": "Mc Elroy/Kremmli", "provider": "METAR", "latitude": 40.05361, "longitude": -106.36889, "code": "K20V", "title": "Mc+Elroy/Kremmli+%28METAR%29+7412+ft", "elevation": 7412},
    {"name": "SH-9 Sum/Grand CO L", "provider": "CDOT", "latitude": 39.91342, "longitude": -106.32601, "code": "CO140", "title": "SH-9+Sum%2FGrand+CO+LN+%28CDOT%29+7797+ft", "elevation": 7797},
    {"name": "Elliot Ck Canal nr", "provider": "MesoWest", "latitude": 39.87361, "longitude": -106.33056, "code": "ELTC2", "title": "Elliot+Ck+Canal+nr+Green+Mountai+%28MesoWest%29+7995+ft", "elevation": 7995},
    {"name": "Beaver Ck Village", "provider": "SNOTEL", "latitude": 39.59917, "longitude": -106.51142, "code": "BCVC2", "title": "Beaver+Ck+Village+%28SNOTEL%29+8500+ft", "elevation": 8500},
    {"name": "CW9561 Kremmling", "provider": "APRSWXNET", "latitude": 40.1395, "longitude": -106.5312, "code": "C9561", "title": "CW9561+Kremmling+%28APRSWXNET%29+8626+ft", "elevation": 8626},
    {"name": "0.5 mi E of Vail -", "provider": "CDOT", "latitude": 39.62122, "longitude": -106.27850, "code": "CO160", "title": "0.5+mi+E+of+Vail+-+RTR+%28CDOT%29+8776+ft", "elevation": 8776},
    {"name": "Middle Fork Camp", "provider": "SNOTEL", "latitude": 39.79560, "longitude": -106.02730, "code": "MFKC2", "title": "Middle+Fork+Camp+%28SNOTEL%29+8940+ft", "elevation": 8940},
    {"name": "Dillon 1E", "provider": "MesoWest", "latitude": 39.62611, "longitude": -106.03556, "code": "DLLC2", "title": "Dillon+1E+%28MesoWest%29+9065+ft", "elevation": 9065},
    {"name": "Dowd Junction", "provider": "RAWS", "latitude": 39.62764, "longitude": -106.45217, "code": "DJTC2", "title": "Dowd+Junction+%28RAWS%29+9068+ft", "elevation": 9068},
    {"name": "0.9 mi W of CO-9 Fr", "provider": "CDOT", "latitude": 39.58391, "longitude": -106.10911, "code": "CO153", "title": "0.9+mi+W+of+CO-9+Frisco+%28CDOT%29+9134+ft", "elevation": 9134},
    {"name": "1.3 mi W of Frisco", "provider": "CDOT", "latitude": 39.56317, "longitude": -106.12958, "code": "CO155", "title": "1.3+mi+W+of+Frisco+%28CDOT%29+9256+ft", "elevation": 9256},
    {"name": "Keystone", "provider": "PWS", "latitude": 39.60992, "longitude": -105.95394, "code": "CAKEY", "title": "Keystone+%28PWS%29+9370+ft", "elevation": 9370},
    {"name": "2.4 mi E of Silvert", "provider": "CDOT", "latitude": 39.64654, "longitude": -106.02644, "code": "CO158", "title": "2.4+mi+E+of+Silverthorne+%28CDOT%29+9390+ft", "elevation": 9390},
    {"name": "Summit Ranch", "provider": "SNOTEL", "latitude": 39.71796, "longitude": -106.15802, "code": "SUMC2", "title": "Summit+Ranch+%28SNOTEL%29+9400+ft", "elevation": 9400},
    {"name": "Mccoy Park", "provider": "SNOTEL", "latitude": 39.60468, "longitude": -106.54128, "code": "MCYC2", "title": "Mccoy+Park+%28SNOTEL%29+9480+ft", "elevation": 9480},
    {"name": "0.6 mi E of CO-91 C", "provider": "CDOT", "latitude": 39.51496, "longitude": -106.14546, "code": "CO154", "title": "0.6+mi+E+of+CO-91+Copper+Mtn+%28CDOT%29+9668+ft", "elevation": 9668},
    {"name": "2.9 mi W of EJMT -", "provider": "CDOT", "latitude": 39.66206, "longitude": -105.98276, "code": "CO159", "title": "2.9+mi+W+of+EJMT+-+The+Box+%28CDOT%29+10208+ft", "elevation": 10208},
    {"name": "2.1 mi W of Vail Pa", "provider": "CDOT", "latitude": 39.56594, "longitude": -106.23739, "code": "CO172", "title": "2.1+mi+W+of+Vail+Pass+Summit+%28CDOT%29+10214+ft", "elevation": 10214},
    {"name": "Breckenridge 5S", "provider": "HADS", "latitude": 39.40861, "longitude": -106.04583, "code": "BKRC2", "title": "Breckenridge+5S+%28HADS%29+10230+ft", "elevation": 10230},
    {"name": "Vail Pass - CDOT Ya", "provider": "CAIC", "latitude": 39.57861, "longitude": -106.24722, "code": "CAVLP", "title": "Vail+Pass+-+CDOT+Yard+%28CAIC%29+10250+ft", "elevation": 10250},
    {"name": "Vail Mountain", "provider": "SNOTEL", "latitude": 39.61676, "longitude": -106.38006, "code": "VLMC2", "title": "Vail+Mountain+%28SNOTEL%29+10300+ft", "elevation": 10300},
    {"name": "Vail SA - Mid-Mtn", "provider": "VailResort", "latitude": 39.61751, "longitude": -106.38004, "code": "CAVMM", "title": "Vail+SA+-+Mid-Mtn+%28VailResort%29+10303+ft", "elevation": 10303},
    {"name": "Elliot Ridge", "provider": "SNOTEL", "latitude": 39.86400, "longitude": -106.42460, "code": "ELRC2", "title": "Elliot+Ridge+%28SNOTEL%29+10520+ft", "elevation": 10520},
    {"name": "Copper Mountain", "provider": "SNOTEL", "latitude": 39.48954, "longitude": -106.17095, "code": "CPMC2", "title": "Copper+Mountain+%28SNOTEL%29+10550+ft", "elevation": 10550},
    {"name": "Vail Pass", "provider": "CDOT", "latitude": 39.52958, "longitude": -106.21687, "code": "CO070", "title": "Vail+Pass+%28CDOT%29+10582+ft", "elevation": 10582},
    {"name": "1.7 mi W of EJMT -", "provider": "CDOT", "latitude": 39.67405, "longitude": -105.96589, "code": "CO157", "title": "1.7+mi+W+of+EJMT+-+Upper+RTR+%28CDOT%29+10610+ft", "elevation": 10610},
    {"name": "Vail SA - China Bow", "provider": "VailResort", "latitude": 39.59772, "longitude": -106.33835, "code": "CAVCB", "title": "Vail+SA+-+China+Bowl+%28VailResort%29+11031+ft", "elevation": 11031},
    {"name": "Grizzly Peak", "provider": "SNOTEL", "latitude": 39.64631, "longitude": -105.86973, "code": "GZPC2", "title": "Grizzly+Peak+%28SNOTEL%29+11100+ft", "elevation": 11100},
    {"name": "Vail SA - Blue Sky", "provider": "VailResort", "latitude": 39.57106, "longitude": -106.33239, "code": "CAVBS", "title": "Vail+SA+-+Blue+Sky+%28VailResort%29+11109+ft", "elevation": 11109},
    {"name": "EJT West Portal", "provider": "CDOT", "latitude": 39.67626, "longitude": -105.94356, "code": "CO126", "title": "EJT+West+Portal+%28CDOT%29+11126+ft", "elevation": 11126},
    {"name": "Ptarmigan", "provider": "USGS", "latitude": 39.49841944, "longitude": -106.2734778, "code": "USPTA", "title": "Ptarmigan+%28USGS%29+11202+ft", "elevation": 11202},
    {"name": "Vail SA - PHQ", "provider": "VailResort", "latitude": 39.60548, "longitude": -106.35657, "code": "CAVPQ", "title": "Vail+SA+-+PHQ+%28VailResort%29+11248+ft", "elevation": 11248},
    {"name": "Boss Basin", "provider": "USGS", "latitude": 39.47235278, "longitude": -106.2628, "code": "USBBN", "title": "Boss+Basin+%28USGS%29+11259+ft", "elevation": 11259},
    {"name": "Fremont Pass", "provider": "CDOT", "latitude": 39.36726, "longitude": -106.18777, "code": "CO121", "title": "Fremont+Pass+%28CDOT%29+11318+ft", "elevation": 11318},
    {"name": "Hoosier Pass", "provider": "SNOTEL", "latitude": 39.36055, "longitude": -106.05994, "code": "HOOC2", "title": "Hoosier+Pass+%28SNOTEL%29+11400+ft", "elevation": 11400},
    {"name": "Fremont Pass", "provider": "SNOTEL", "latitude": 39.37991, "longitude": -106.19681, "code": "FMTC2", "title": "Fremont+Pass+%28SNOTEL%29+11400+ft", "elevation": 11400},
    {"name": "A-Basin SA-MidMtn", "provider": "A-BasinSA", "latitude": 39.63227, "longitude": -105.86910, "code": "CAABM", "title": "A-Basin+SA-MidMtn+%28A-BasinSA%29+11660+ft", "elevation": 11660},
    {"name": "A-Basin SA-Pali", "provider": "A-BasinSA", "latitude": 39.63334, "longitude": -105.87870, "code": "CAABP", "title": "A-Basin+SA-Pali+%28A-BasinSA%29+11920+ft", "elevation": 11920},
    {"name": "Copper Mountain", "provider": "METAR", "latitude": 39.47523, "longitude": -106.15228, "code": "KCCU", "title": "Copper+Mountain+%28METAR%29+12074+ft", "elevation": 12074},
    {"name": "A-Basin SA-Summit", "provider": "A-BasinSA", "latitude": 39.62455, "longitude": -105.87630, "code": "CAABT", "title": "A-Basin+SA-Summit+%28A-BasinSA%29+12462+ft", "elevation": 12462}
]

# Forecast locations
forecast_locations = [
    {
        "name": "A-Basin Ski Area",
        "location_code": "FWAB",
        "latitude": 39.6324,
        "longitude": -105.8690,
        "model_elevation": 11066,
        "forecast_image_urls": {
            "WRF": "https://looper.avalanche.state.co.us/weather/ptfcst/wrf/current/FWAB-e.png",
            "WRFHR": "https://looper.avalanche.state.co.us/weather/ptfcst/wrfhr/current/FWAB-e.png",
            "NAM": "https://looper.avalanche.state.co.us/weather/ptfcst/nam/current/FWAB-e.png"
        }
    },
    {
        "name": "Beaver Creek Ski Area",
        "location_code": "BC_TOP",
        "latitude": 39.5673,
        "longitude": -106.5076,
        "model_elevation": 10862,
        "forecast_image_urls": {
            "WRF": "https://looper.avalanche.state.co.us/weather/ptfcst/wrf/current/BC_TOP-e.png",
            "WRFHR": "https://looper.avalanche.state.co.us/weather/ptfcst/wrfhr/current/BC_TOP-e.png",
            "NAM": "https://looper.avalanche.state.co.us/weather/ptfcst/nam/current/BC_TOP-e.png"
        }
    },
    {
        "name": "Breckenridge Ski Area",
        "location_code": "BRECKENRDG",
        "latitude": 39.4727,
        "longitude": -106.1026,
        "model_elevation": 11587,
        "forecast_image_urls": {
            "WRF": "https://looper.avalanche.state.co.us/weather/ptfcst/wrf/current/BRECKENRDG-e.png",
            "WRFHR": "https://looper.avalanche.state.co.us/weather/ptfcst/wrfhr/current/BRECKENRDG-e.png",
            "NAM": "https://looper.avalanche.state.co.us/weather/ptfcst/nam/current/BRECKENRDG-e.png"
        }
    },
    {
        "name": "Copper Mt Ski Area",
        "location_code": "COPPERMTN",
        "latitude": 39.4843,
        "longitude": -106.1555,
        "model_elevation": 10724,
        "forecast_image_urls": {
            "WRF": "https://looper.avalanche.state.co.us/weather/ptfcst/wrf/current/COPPERMTN-e.png",
            "WRFHR": "https://looper.avalanche.state.co.us/weather/ptfcst/wrfhr/current/COPPERMTN-e.png",
            "NAM": "https://looper.avalanche.state.co.us/weather/ptfcst/nam/current/COPPERMTN-e.png"
        }
    },
    {
        "name": "Keystone Ski Area",
        "location_code": "KEYSTONE",
        "latitude": 39.5792,
        "longitude": -105.9429,
        "model_elevation": 10869,
        "forecast_image_urls": {
            "WRF": "https://looper.avalanche.state.co.us/weather/ptfcst/wrf/current/KEYSTONE-e.png",
            "WRFHR": "https://looper.avalanche.state.co.us/weather/ptfcst/wrfhr/current/KEYSTONE-e.png",
            "NAM": "https://looper.avalanche.state.co.us/weather/ptfcst/nam/current/KEYSTONE-e.png"
        }
    },
    {
        "name": "Loveland Pass",
        "location_code": "LVLNDPASS",
        "latitude": 39.6637,
        "longitude": -105.8788,
        "model_elevation": 11856,
        "forecast_image_urls": {
            "WRF": "https://looper.avalanche.state.co.us/weather/ptfcst/wrf/current/LVLNDPASS-e.png",
            "WRFHR": "https://looper.avalanche.state.co.us/weather/ptfcst/wrfhr/current/LVLNDPASS-e.png",
            "NAM": "https://looper.avalanche.state.co.us/weather/ptfcst/nam/current/LVLNDPASS-e.png"
        }
    },
    {
        "name": "Vail Pass",
        "location_code": "VAIL_PASS",
        "latitude": 39.5310,
        "longitude": -106.2170,
        "model_elevation": 11016,
        "forecast_image_urls": {
            "WRF": "https://looper.avalanche.state.co.us/weather/ptfcst/wrf/current/VAIL_PASS-e.png",
            "WRFHR": "https://looper.avalanche.state.co.us/weather/ptfcst/wrfhr/current/VAIL_PASS-e.png",
            "NAM": "https://looper.avalanche.state.co.us/weather/ptfcst/nam/current/VAIL_PASS-e.png"
        }
    },
    {
        "name": "Vail Ski Area",
        "location_code": "VAILSKI",
        "latitude": 39.5988,
        "longitude": -106.3757,
        "model_elevation": 10190,
        "forecast_image_urls": {
            "WRF": "https://looper.avalanche.state.co.us/weather/ptfcst/wrf/current/VAILSKI-e.png",
            "WRFHR": "https://looper.avalanche.state.co.us/weather/ptfcst/wrfhr/current/VAILSKI-e.png",
            "NAM": "https://looper.avalanche.state.co.us/weather/ptfcst/nam/current/VAILSKI-e.png"
        }
    }
]

# Webcam Locations
webcam_locations = {
    "A-Basin Zuma Bowl": {
        "latitude": 39.62271,
        "longitude": -105.87304,
        "image_url": "https://ftp.purchus.io/wwwroot/abasin/webcam/abasincam5.jpg"
    }
}

## 6. Run to create an html file

In [7]:
# Create the enhanced map
create_map = weather_map(stations, forecast_locations, webcam_locations)

# Save the map to an HTML file
create_map.save("weather_map.html")

print("Weather Map has been created and saved as 'weather_map.html'")

Weather Map has been created and saved as 'weather_map.html'


## 7. Test map in notebook

In [8]:
create_map